## SLSQP Model for Deconvolution of Humon Immune Cell Types
Autor: Sumeyye Su


$$ min_A(||AS-X||^2), s.t. \bigg\{ \sum_{i=1}^{N} a_{ki}=1 , a_{ki}\geq0, $$

$$x_{ij}=\sum_{i=1}^{N} a_{ki}s_{ij}$$


x_ij: g ene expression level of gene j in a sample k 

a_ki: i cell type proportion for sample k 

s_ij: gene expression level of gene j in i cell type
 
 More generally, matrix form of the problem is:
$$ X=AS$$
X: mixture data 

A: proportion matrix ( desired matrix )

S: signature matrix 

In [21]:
import os
import pandas as pd
import seaborn as sns
import matplotlib.pylab as plt
import numpy as np
from scipy.optimize import minimize
from sklearn.metrics import r2_score
from scipy.spatial import distance
from scipy.optimize import Bounds
sample_mixture =pd.read_csv('sample_mixture.csv')
sample_mixture=sample_mixture.set_index('Unnamed: 0')
sample_signature=pd.read_csv('sample_signature.csv')
sample_signature=sample_signature.set_index('Unnamed: 0')

sample_signature.head()

,ENSG00000074800.13,ENSG00000142657.20,ENSG00000028137.18,ENSG00000142676.12,ENSG00000169442.8,ENSG00000198830.10,ENSG00000000938.12,ENSG00000130775.15,ENSG00000162511.7,ENSG00000182866.16,...,ENSG00000212907.2,ENSG00000198886.2,ENSG00000198786.2,ENSG00000198695.2,ENSG00000198727.2,ENSG00000210195.2,ENSG00000210196.2,ERCC-00002,ERCC-00074,ERCC-00096
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
CD8_naive,0.019306,0.001534,0.000521,0.029999,0.012729,0.016410,0.000091,0.000971,0.031175,0.030230,...,0.426865,0.245647,0.108467,0.075782,0.289643,0.006568,0.017030,0.083341,0.069839,0.053507
CD8_CM,0.019509,0.001310,0.002110,0.024920,0.016353,0.017403,0.000640,0.001060,0.034476,0.031658,...,0.297374,0.158015,0.089361,0.052855,0.232452,0.005435,0.010333,0.074055,0.067231,0.045749
CD8_EM,0.024111,0.001771,0.005592,0.011015,0.017618,0.023779,0.010812,0.001883,0.040473,0.041626,...,0.362777,0.206895,0.099897,0.086693,0.259394,0.008106,0.009930,0.148139,0.114487,0.107721
CD8_TE,0.014170,0.001175,0.003650,0.011725,0.016800,0.012720,0.012201,0.001134,0.023218,0.023797,...,0.254792,0.137287,0.081357,0.065394,0.212041,0.004548,0.007840,0.098993,0.079170,0.064564
MAIT,0.018945,0.001052,0.001996,0.016921,0.011174,0.012615,0.001082,0.001191,0.036514,0.026527,...,0.376352,0.202431,0.105858,0.064155,0.277510,0.006607,0.009190,0.092059,0.056347,0.060531


In [22]:
sample_mixture.head()

,CYFZ_PBMC,FY2H_PBMC,FLWA_PBMC,453W_PBMC,684C_PBMC,CZJE_PBMC,DZQV_PBMC,925L_PBMC,9JD4_PBMC,G4YW_PBMC,4DUY_PBMC,36TS_PBMC,CR3L_PBMC
Unnamed: 0,,,,,,,,,,,,,
ENSG00000074800.13,0.014361,0.017776,0.024915,0.028890,0.019606,0.019449,0.019360,0.019842,0.017690,0.019059,0.023658,0.026257,0.016431
ENSG00000142657.20,0.003672,0.004931,0.006371,0.007863,0.004120,0.005371,0.006524,0.007645,0.005231,0.006408,0.007588,0.008252,0.004536
ENSG00000028137.18,0.003784,0.006577,0.008855,0.011248,0.005630,0.003506,0.004894,0.008066,0.007808,0.008795,0.010685,0.006361,0.011960
ENSG00000142676.12,0.011220,0.005002,0.016300,0.011613,0.006747,0.011703,0.017182,0.020796,0.020962,0.017967,0.017801,0.018329,0.009652
ENSG00000169442.8,0.010804,0.010508,0.034717,0.016382,0.011357,0.008059,0.011397,0.012659,0.010196,0.011062,0.010266,0.007695,0.006545


In [44]:
def optimization_SLSQP(mixture,signature,c_n,x0):
    'main function to find the proportion matrix'
    def func(x,g,s):
        return distance.euclidean(np.matmul(x,s),g)
    cell_type_data=pd.DataFrame()
    for column in mixture:
        eq_cons = {'type': 'eq',
            'fun' : lambda x: np.array([1-(np.sum(x))])}

        bounds = Bounds(np.zeros(c_n), np.ones(c_n))

    
        res = minimize(func, x0, method='SLSQP', 
                constraints=[eq_cons], options={'ftol': 1e-30, 'disp': False},
                bounds=bounds,args=(mixture[column],signature))
        result=res.x
        cell_type=pd.DataFrame(result,columns=[column],index=[signature.index.values])
        cell_type_data=pd.concat([cell_type_data,cell_type],axis=1)
    
    return cell_type_data





In [25]:
a=optimization_SLSQP(sample_mixture,sample_signature,29,np.array([0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1]))
a

,CYFZ_PBMC,FY2H_PBMC,FLWA_PBMC,453W_PBMC,684C_PBMC,CZJE_PBMC,DZQV_PBMC,925L_PBMC,9JD4_PBMC,G4YW_PBMC,4DUY_PBMC,36TS_PBMC,CR3L_PBMC
CD8_naive,1.540206e-17,0.000000e+00,0.000000e+00,3.646152e-16,9.578456e-17,8.442897e-17,5.174213e-18,1.912797e-19,4.448204e-17,0.000000e+00,1.197645e-16,8.789408e-21,0.000000e+00
CD8_CM,2.414467e-16,1.238756e-16,2.206365e-16,0.000000e+00,1.531274e-16,0.000000e+00,1.996018e-16,2.931499e-03,2.296206e-18,4.780286e-17,5.251387e-17,9.329314e-17,0.000000e+00
CD8_EM,1.120461e-15,0.000000e+00,2.732177e-01,2.593679e-01,3.126233e-01,0.000000e+00,1.577587e-16,6.799820e-17,4.238869e-18,0.000000e+00,4.756881e-24,7.294887e-17,2.694245e-16
CD8_TE,2.357753e-01,1.547602e-02,0.000000e+00,0.000000e+00,1.558542e-02,4.197724e-01,0.000000e+00,2.332694e-16,3.244552e-17,4.947922e-17,6.072929e-17,3.774099e-23,2.165143e-01
MAIT,1.235285e-16,0.000000e+00,7.550054e-17,2.031340e-16,1.862390e-16,0.000000e+00,9.086529e-17,5.037203e-17,2.448343e-17,9.080038e-17,1.831832e-17,2.885036e-02,3.023746e-16
VD2+,1.786905e-16,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,9.521722e-19,1.183053e-18,6.042881e-17,1.649023e-17,5.648484e-18,1.360781e-16,0.000000e+00
VD2-,2.528529e-16,0.000000e+00,0.000000e+00,1.432492e-17,1.458193e-16,7.399559e-17,3.214107e-17,8.334857e-03,2.069117e-16,7.057341e-02,2.958754e-02,5.136969e-17,1.984265e-16
TFH,8.416009e-17,2.780938e-16,1.694894e-16,1.732601e-16,1.202690e-16,2.361163e-17,3.570510e-17,1.030373e-18,3.894288e-17,1.730319e-16,9.182748e-17,1.521947e-19,0.000000e+00
Treg,0.000000e+00,2.128316e-16,0.000000e+00,0.000000e+00,2.919345e-18,3.072198e-17,1.738511e-17,1.782933e-16,8.729903e-19,8.942852e-18,4.015833e-17,7.963301e-17,1.199710e-16
Th1/Th17,1.594831e-16,0.000000e+00,7.521989e-17,0.000000e+00,6.906359e-17,0.000000e+00,2.289049e-23,2.539216e-16,3.197640e-17,0.000000e+00,6.324790e-17,1.608962e-20,1.519493e-16


In [30]:
def generate_test_data(samples,cell_type, gene,signature_matrix):
    test_proportion_data=np.empty(shape=[samples,cell_type])
    test_mixture=np.empty(shape=[samples,gene])
    test_mixture_df=pd.DataFrame(index= signature_matrix.columns.values.tolist())
    for i in range(samples):
        test_proportion_data[i]=np.array(np.random.random(cell_type))
        test_proportion_data[i] /=np.sum(test_proportion_data[i])
        test_mixture[i]=np.matmul(test_proportion_data[i],signature_matrix)
        test_mixture_df[i]=pd.DataFrame(test_mixture[i],index= signature_matrix.columns.values.tolist())
        test_mixture_df=test_mixture_df.fillna(0)
    return(test_mixture_df,test_proportion_data)   

In [35]:
test_mixture,test_proportion=generate_test_data(10,29,500,sample_signature)
test_proportion

array([[0.04027532, 0.0412057 , 0.05611429, 0.03425339, 0.01550923,
        0.05263042, 0.04763827, 0.04828912, 0.00212888, 0.03640467,
        0.04687398, 0.00246757, 0.04044454, 0.00049561, 0.05700408,
        0.02125315, 0.02066081, 0.05680129, 0.04493187, 0.027096  ,
        0.02573295, 0.02846598, 0.04868884, 0.04580539, 0.04614304,
        0.01512445, 0.0213223 , 0.02360759, 0.05263127],
       [0.02657656, 0.01719663, 0.01756489, 0.02855185, 0.03418211,
        0.02124236, 0.06210357, 0.06071568, 0.05587667, 0.06070954,
        0.00273811, 0.01769243, 0.03098022, 0.01857938, 0.02481902,
        0.04555264, 0.05464273, 0.0573125 , 0.06227755, 0.00017855,
        0.03753279, 0.03935996, 0.02177802, 0.04650585, 0.00642209,
        0.06417556, 0.03549669, 0.02983334, 0.01940271],
       [0.06548728, 0.05631864, 0.04939175, 0.03138807, 0.01951302,
        0.02231753, 0.02489543, 0.06372733, 0.03291319, 0.03112592,
        0.02810368, 0.04462702, 0.00475871, 0.05257844, 0.00211339,
  

In [36]:
test_proportion_result=optimization_SLSQP(test_mixture,sample_signature,29,np.array([0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1]))
test_proportion_result=test_proportion_result.T
test_proportion_result.head()

,CD8_naive,CD8_CM,CD8_EM,CD8_TE,MAIT,VD2+,VD2-,TFH,Treg,Th1/Th17,...,C_mono,I_mono,NC_mono,NK,pDC,mDC,Neutrophils,Basophils,CD4_TE,Th1
0,0.040275,0.041206,0.056114,0.034254,0.015509,0.052630,0.047638,0.048289,0.002129,0.036405,...,0.027096,0.025733,0.028466,0.048689,0.045805,0.046143,0.015124,0.021322,0.023607,0.052631
1,0.026576,0.017197,0.017565,0.028552,0.034182,0.021242,0.062104,0.060715,0.055877,0.060711,...,0.000178,0.037533,0.039360,0.021778,0.046506,0.006422,0.064176,0.035497,0.029833,0.019403
2,0.065487,0.056319,0.049392,0.031388,0.019513,0.022317,0.024896,0.063726,0.032913,0.031127,...,0.051005,0.002513,0.044451,0.049233,0.018041,0.057380,0.008753,0.028851,0.058076,0.040502
3,0.062603,0.043677,0.051317,0.056534,0.046741,0.039411,0.005390,0.022670,0.040337,0.005552,...,0.054562,0.036876,0.015999,0.025405,0.026984,0.050911,0.018635,0.056878,0.057522,0.016956
4,0.011249,0.030690,0.019204,0.014202,0.014259,0.042531,0.018246,0.021034,0.046224,0.047060,...,0.008891,0.032597,0.005777,0.055405,0.059224,0.037991,0.068797,0.023835,0.042099,0.007786


In [37]:
xx=r2_score(test_proportion,test_proportion_result)
xx

0.9999999994010883

In [38]:
def generate_test_data_with_score(samples,cell_type, gene, mixture_data,signature_matrix,x0):
    '''
    This function gives us the scores of randomly generated mixture data sets
       
       sample is # of samples
       cell_type # of cell_types
       gene is # of gene in the signature matrix
       mixture data: real threshold mixture data
       signature data : threshold signature matrix'''
    
    test_proportion_data=np.empty(shape=[samples,cell_type])
    test_mixture=np.empty(shape=[samples,gene])
    test_mixture_df=pd.DataFrame(index= mixture_data.index)
    result_test=pd.DataFrame(index=signature_matrix.columns.values.tolist(),columns=range(samples))
    score_data=np.empty(shape=[samples,1])
    for i in range(samples):
        test_proportion_data[i]=np.array(np.random.random(cell_type))
        test_proportion_data[i] /=np.sum(test_proportion_data[i])
        test_mixture[i]=np.matmul(test_proportion_data[i],signature_matrix)
        test_mixture_df[i]=pd.DataFrame(test_mixture[i],index= mixture_data.index)
        test_mixture_df=test_mixture_df.fillna(0)
        result_test=optimization_SLSQP(test_mixture_df, signature_matrix,cell_type,x0)
        result_test=result_test.T
        score_data[i]=r2_score(test_proportion_data[i],result_test.iloc[i])
    return score_data


In [39]:
'an example for using generate_test_data_score function to find average score of randomly generated mixture databy using above sample mixture and signature matrix'''

a=generate_test_data_score(50,29,500,sample_mixture,sample_signature,np.array([0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1]))


In [43]:
average_score=np.mean(a)
average_score

0.9999999994458981